### Install dependancies

In [53]:
%pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ reservoirpy

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple/
  Obtaining dependency information for reservoirpy from https://test-files.pythonhosted.org/packages/e4/54/21e451f81c2523472a162e97278db2cd5db4b44e584876add12ec6629ae7/reservoirpy-0.3.10-py3-none-any.whl.metadata
  Obtaining dependency information for dill>=0.3.1.1 from https://files.pythonhosted.org/packages/f5/3a/74a29b11cf2cdfcd6ba89c0cecd70b37cd1ba7b77978ce611eb7a146a832/dill-0.3.7-py3-none-any.whl.metadata
  Using cached dill-0.3.7-py3-none-any.whl.metadata (9.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [54]:
%pip install torch torchvision torchaudio transformers


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Load wav2vec2 model

In [14]:
from transformers import Wav2Vec2Model, Wav2Vec2Tokenizer
import librosa
import torch

# Charger le modèle pré-entraîné et le tokenizer
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

/Users/naowak/Thesis/code/tchatche/venv_tchatche/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
(…)2vec2-base-960h/resolve/main/config.json: 100%|██████████| 1.60k/1.60k [00:00<00:00, 4.22MB/s]
model.safetensors: 100%|██████████| 378M/378M [00:26<00:00, 14.1MB/s] 
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
(…)v2vec2-base-960h/resolve/main/vocab.json: 100%|██████████| 291/291 [00:00<00:00, 1.30MB/s]
(…)60h/resolve/main/special_t

### Compute all segments of 5 seconds

In [20]:
import os
import librosa
import soundfile as sf
import pandas as pd

def split_audio_files(filenames, ratings, folder_path, output_folder, target_length=5.0):
    """
    Découpe tous les fichiers audio dans le dossier spécifié en segments de 5 secondes.
    Les segments inférieurs à 5 secondes sont ignorés.

    :param folder_path: Chemin vers le dossier contenant les fichiers audio.
    :param target_length: Longueur cible pour les segments, en secondes.
    """
    for i, file_name in enumerate(filenames):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)
            audio, sr = librosa.load(file_path, sr=None)

            # Calculer le nombre de samples pour 5 secondes
            target_samples = int(target_length * sr)

            # Découper en segments de 5 secondes
            for start in range(0, len(audio), target_samples):
                end = start + target_samples

                # Ignorer les segments de moins de 5 secondes
                if end <= len(audio):
                    segment = audio[start:end]
                    segment_file_name = f"{file_name[:-4]}_segment_{start // target_samples}_r{ratings[i]}.wav"
                    segment_file_path = os.path.join(output_folder, segment_file_name)

                    # Enregistrer le segment
                    sf.write(segment_file_path, segment, sr)

# Read the csv file containing the filenames and ratings
df_source = pd.read_csv('./data-csv/tchatche_split_x.csv')

# Utiliser la fonction
split_audio_files(df_source['file_name'], df_source['rating'], 'raw-audio', 'raw-audio-segments')


### Load data and extract features from wav2vec2

In [29]:
import pandas as pd 
import os
from tqdm import tqdm

# Define the directory path
dir_path = './raw-audio-segments'

# Retrieve all filenames in the directory
filenames = os.listdir(dir_path)

# all_features
data = []

# Loop through the list of .wav files and compute features from MFCC for each file
for filename in tqdm(filenames):

    # Get rating from filename
    rating = int(filename.split('_')[-1][1])

    # Charger et préparer l'audio (exemple)
    audio_input, sampling_rate = librosa.load(os.path.join(dir_path, filename), sr=16000)

    # Tokenize et extraire les features
    input_values = tokenizer(audio_input, return_tensors="pt", padding="longest").input_values
    with torch.no_grad():
        features = model(input_values).last_hidden_state
    
    data.append({
        'file_name': filename,
        'rating': rating,
        'features': features
    })
    
    # Append features to list
    line = {'file_name': filename, 'rating': rating, 'features': features}
    data.append(line)

# Create a dataframe from the list of features
df = pd.DataFrame(data)
df.head(2)

# Save the dataframe to a csv file
df.to_csv('./data-csv/tchatche_split_wav2vec2_features.csv', index=False)

100%|██████████| 1624/1624 [03:38<00:00,  7.45it/s]


,file_name,rating,features
0,QG_20230530144858_1_2818740_3060040_9657_12008...,5,"[[[tensor(-0.0926), tensor(-0.0106), tensor(-0..."
1,QG_20230530144858_1_2818740_3060040_9657_12008...,5,"[[[tensor(-0.0926), tensor(-0.0106), tensor(-0..."


### Attempt with reservoirpy

In [51]:
from sklearn.model_selection import train_test_split
import numpy as np

data = []
ratings = []
for i, row in df.iterrows():
    ratings += [row['rating'] for _ in range(row['features'].shape[1])]
    data += [row['features'].squeeze().numpy()]

data = np.array(data)
ratings = np.array(ratings).reshape(-1, data.shape[1])
print(data.shape, ratings.shape)

X_train, X_test, y_train, y_test = train_test_split(data, ratings, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3248, 249, 768) (3248, 249)
(2598, 249, 768) (650, 249, 768) (2598, 249) (650, 249)


In [103]:
from reservoirpy.nodes import Reservoir, Ridge, Input, ESN

#data = Input(input_dim=1)
reservoir = Reservoir(4000, lr=0.2, sr=1.1)
readout = Ridge(ridge=1e-8)
esn = ESN(reservoir=reservoir, readout=readout, workers=-1)

In [104]:
forecast = esn.fit(X_train, y_train.reshape(-1, 249, 1))

Running ESN-1:  22%|██▏       | 580/2598 [00:44<02:35, 13.00it/s]/Users/naowak/Thesis/code/tchatche/venv_tchatche/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
Running ESN-1: 100%|██████████| 2598/2598 [03:22<00:00, 12.83it/s]


Fitting node ESN-1...


In [105]:
from sklearn.metrics import mean_squared_error

y_pred = esn.run(X_test)
y_pred = np.array(y_pred)
y_pred = y_pred.reshape(y_pred.shape[0], y_pred.shape[1])
mean_squared_error(y_pred.mean(axis=1), y_test.mean(axis=1))

Running ESN-1: 100%|██████████| 650/650 [00:37<00:00, 17.19it/s]


3.104050652209672

## Recherche d'hyper-paramètres


In [107]:

def objective(dataset, config, *, iss, N, sr, lr, ridge):
    dataset = ((X_train, y_train), (X_test, y_test))

    reservoir = Reservoir(N, lr=lr, sr=sr, input_scaling=iss)
    readout = Ridge(ridge=ridge)
    esn = ESN(reservoir=reservoir, readout=readout, workers=-1)

    esn.fit(X_train, y_train.reshape(-1, 249, 1))

    y_pred = np.array(esn.run(X_test))
    y_pred = y_pred.reshape(y_pred.shape[0], y_pred.shape[1])
    return {"loss":mean_squared_error(y_pred.mean(axis=1), y_test.mean(axis=1))}
    

In [115]:
hyperopt_config = {
    "exp": "hyperopt-bert-1", # the experimentation name
    "hp_max_evals": 200,             # the number of differents sets of parameters hyperopt has to try
    "hp_method": "random",           # the method used by hyperopt to chose those sets (see below)
    "seed": 42,                      # the random state seed, to ensure reproducibility
    "instances_per_trial": 1,        # how many random ESN will be tried with each sets of parameters
    "hp_space": {                    # what are the ranges of parameters explored
        "N": ["choice", 1500],             # the number of neurons is fixed to 500
        "sr": ["loguniform", 1e-3, 100],   # the spectral radius is log-uniformly distributed between 1e-2 and 10
        "lr": ["loguniform", 1e-3, 1],  # idem with the leaking rate, from 1e-3 to 1
        "iss": ["loguniform", 1e-2, 1e1],           # the input scaling is fixed
        "ridge": ["loguniform", 1e-8, 1e-3],        # and so is the regularization parameter.
    }
}


import json

# we precautionously save the configuration in a JSON file
# each file will begin with a number corresponding to the current experimentation run number.
with open(f"{hyperopt_config['exp']}.config.json", "w+") as f:
    json.dump(hyperopt_config, f)

In [113]:
%pip install hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 14.1 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for cloudpickle from https://files.pythonhosted.org/packages/96/43/dae06432d0c4b1dc9e9149ad37b4ca8384cf6eb7700cd9215b177b914f0a/cloudpickle-3.0.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 12.4 MB/s eta 0:00:00
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492024 sha256=d8b01685f982779876b8f4a1b334232a5f733e1ec65629ca7f9651ab85bfb5dc
  Stored in directory: /Users/naowak/Library/Caches/pip/wheels/da/19/ca/9d8c44cd311a955509d7e13da3f0bea42400c469ef825b580b
Successfully built future

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install

In [116]:
from reservoirpy.hyper import research
from reservoirpy import verbosity
verbosity(0)
dataset = (X_train[:500, :, :], y_train[:500, :]), (X_test[:150, :, :], y_test[:150, :])

best = research(objective, dataset, f"{hyperopt_config['exp']}.config.json", ".")

 46%|████▌     | 91/200 [1:30:38<1:48:34, 59.76s/trial, best loss: 2.6167795378239616]


KeyboardInterrupt: 